In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'dvd'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 10642 palavras!


Separa os dados em treinamento e teste:

In [2]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

#X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :]
Ytrain, Yval = Y2[train_index], Y2[test_index]

Seleciona features com método do chi-quadrado (baseado nos dados de treinamento):

In [3]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

In [4]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
print(new_vocabulary)

Número de features antes do chi-quadrado: 10642
----------------------------------------
Número de features após chi-quadrado: 713
['grazia', 'rural', 'hollow', 'mckean', 'immort', 'find', 'woodward', 'fourth', 'woo', 'host', 'ethic', ('not', 'seen'), 'freestyl', 'raunchi', 'briefcas', 'arrest', 'hate', 'manipul', 'sadi', 'respond', 'robert', ('not', 'help'), 'ridicul', 'misfram', 'dismal', 'slave', 'sadli', 'local', ('not', 'need'), 'arrau', 'turkish', 'think', 'ultim', 'experi', ('not', 'buy'), ('not', 'expect'), 'devast', ('not', 'use'), 'russel', 'belong', 'anim', ('not', 'scari'), ('not', 'includ'), 'san', 'kurosawa', 'mani', 'molli', 'clive', 'carla', ('not', 'particularli'), 'terribl', 'surpris', ('never', 'get'), 'finest', 'audio', 'gehri', 'novic', 'stray', 'aw', 'win', 'episod', 'thrill', 'trite', 'unforgiv', 'disappoint', ('not', 'better'), 'set', 'broadway', 'bitter', 'splendid', 'vocal', 'unpleas', ('not', 'mind'), ('not', 'make'), 'connelli', 'uninterest', ('not', 'recomm

In [5]:
def calcularProbabilidades(X, Y):
   
    pPos = np.zeros(X.shape[1])
    pNeg = np.zeros(X.shape[1])

    pPos = X[(Y == 1)].sum(axis=0)/sum(Y == 1)
    pNeg = X[(Y == 0)].sum(axis=0)/sum(Y == 0)
    
    
    probPos = sum(Y==1)/len(Y) 
    probNeg = sum(Y==0)/len(Y)
    
    return probPos, probNeg, pPos, pNeg

In [6]:
probP, probN, pPos, pNeg = calcularProbabilidades(Xtrain, Ytrain)

In [7]:
def classificacao(x,probP,probN,pPos,pNeg):

    classe = 0;
    probPos= 0;
    probNeg = 0;

    x = x.toarray()
    probPos = probP * (pPos[(x != 0)]).prod() * (1 - pPos[(x == 0)]).prod()
    probNeg = probN * (pNeg[(x != 0)]).prod() * (1 - pNeg[(x == 0)]).prod()
    
    classe = int(probPos >= probNeg)

    return classe

In [8]:
def predicao(Xval, Yval, probP, probN, pPos, pNeg):
    
    Ypred = []
    for i in range(Xval.shape[0]):
        y = classificacao(Xval[i], probP,probN,pPos,pNeg)
        Ypred.append(y)

    acuracia = np.sum(Ypred==Yval)/len(Yval)
    
    return Ypred, acuracia

In [9]:
Ypred, acuracia = predicao(Xval, Yval, probP, probN, pPos, pNeg)
print("Acurácia é " + str(acuracia*100))

Acurácia é 75.0


In [10]:
classes = np.unique(Y)
auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)


	Revocacao   Precisao   F-medida   Classe
	0.859       0.713      0.779      0
	0.636       0.810      0.713      1
	------------------------------------------------
	0.747       0.762      0.754      Média macro
	0.750       0.750      0.750      Média micro

	Acuracia: 0.750


In [11]:
def curva_aprendizado(X, Y, Xval, Yval):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y)):
        
        probP, probN, pPos, pNeg = calcularProbabilidades(X[:i], Y[:i])
        
        Ypred, acuracia = predicao(X[:i], Y[:i], probP, probN, pPos, pNeg)
        perf_train.append(acuracia)

        Ypred, acuracia = predicao(Xval, Yval, probP, probN, pPos, pNeg)
        perf_val.append(acuracia)


    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()

In [12]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval)

In [13]:
import k_folds as kf
fold = kf.stratified_kfolds(Y, 5, [0, 1])
classes = [0,1]
# cria uma lista vazia para guardar os resultados obtidos em cada fold
resultados = []
k = 1
for train_index, test_index  in fold :
    
    print('\n-----------\n%d-fold: \n-----------\n' % (k) )
    
    Xtrain, Xval = X2[train_index,:], X2[test_index, :]
    Ytrain, Yval = Y2[train_index], Y2[test_index]
    
    Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
    Xval = Xval[:, index]
    
    probP, probN, pPos, pNeg = calcularProbabilidades(Xtrain, Ytrain)
    result, acuracia = predicao(Xval, Yval, probP, probN, pPos, pNeg)
    #print(result)
    #cm = kf.get_confusionMatrix(Yval, result, classes)
    auxResults = anl.relatorioDesempenho(Yval,result, classes, imprimeRelatorio=True)
    resultados.append( auxResults )
    k+=1
    
kf.mediaFolds( resultados, [0,1] )  


-----------
1-fold: 
-----------


	Revocacao   Precisao   F-medida   Classe
	0.866       0.686      0.766      0
	0.596       0.814      0.688      1
	------------------------------------------------
	0.731       0.750      0.740      Média macro
	0.733       0.733      0.733      Média micro

	Acuracia: 0.733

-----------
2-fold: 
-----------


	Revocacao   Precisao   F-medida   Classe
	0.886       0.687      0.774      0
	0.623       0.854      0.721      1
	------------------------------------------------
	0.755       0.771      0.762      Média macro
	0.750       0.750      0.750      Média micro

	Acuracia: 0.750

-----------
3-fold: 
-----------


	Revocacao   Precisao   F-medida   Classe
	0.859       0.676      0.757      0
	0.592       0.810      0.684      1
	------------------------------------------------
	0.726       0.743      0.734      Média macro
	0.725       0.725      0.725      Média micro

	Acuracia: 0.725

-----------
4-fold: 
-----------


	Revocacao   Precisao 